In [2]:
import pandas
import ib_insync as ibi

from datetime import datetime

In [3]:
!python --version

Python 3.7.6


In [4]:
class TimeDouble:
    """
    Basic timestamped data class.
    """
    def __init__(self,
                data_name: str,
                timestamp: datetime,
                value: float)-> None:
        """
        Initialize time double.
        
        Parameters
        ----------
        data_name: str
            Name of the data represented.
        timestamp: datetime
            Time of the realization of the data point.
        value: float
            Numerical value of the data.
        """
        self._data_name = data_name
        self._timestamp = timestamp
        self._value = value
    def get_name(self)-> str:
        return self._data_name
    def get_time(self)-> datetime:
        return self._timestamp
    def get_value(self)-> float:
        return self._value

In [16]:
from collections import deque
from datetime import timedelta
from abc import ABC, abstractmethod
from typing import List
from eventkit import Event

class SignalBase(ABC):
    """
    Base class for signal.
    """
    def __init__(self,
                input_data_array: List[str],
                warmup_length: int,
                signal_name: str)-> None:
        """
        Initialize signal base.
        
        Parameters
        ----------
        input_data_array: List[str]
            List of event subscription that is required for signal calculation.
        warmup_length: int
            Number of data points to 'burn'
        signal_name: str
            Name of the signal.
        """        
        self._input_data_array = input_data_array
        self._signal_name = signal_name
        self._warmup_length = warmup_length
        self._initialize_data_time_storage(input_data_array)
    
    def _initialize_data_time_storage(self, input_data_array: List[str])-> None:
        """
        Initialize storage.
        
        Parameters
        ----------
        input_data_array: List[str]
            List of event subscription that is required for signal calculation.
        """
        data_storage = {}
        time_storage = {}
        for input_name_ in input_data_array:
            data_storage[input_name_] = deque([])
            time_storage[input_name_] = deque([])
        self._data_storage = data_storage
        self._time_storage = time_storage
    
    def check_all_received(self, data_name: str)-> bool:
        output = True
        num_data = len(self._input_data_array)
        if num_data == 1:
            return output # If there is only one incoming data stream, no need to check
        for i in range(num_data):
            event_name_i = self._input_data_array[i]
            if len(self.get_time_by_name(event_name_i)) == 0:
                return False
            
            time_diff = self.get_time_by_name(data_name)[-1] - self.get_time_by_name(event_name_i)[-1]
            if time_diff > timedelta(microseconds = 1):
                return False
        return output
    
    def update_data(self, new_data: TimeDouble)-> None:
        """
        Update data storage
        
        Parameters
        ----------
        new_data: TimeDouble
            Incoming new data.
        """
        # Extend storage
        self._data_storage[new_data.get_name()].append(new_data.get_value())
        self._time_storage[new_data.get_name()].append(new_data.get_time())
        # Remove oldest if warming up is complete
        if len(self._data_storage[new_data.get_name()]) > self._warmup_length:
            self._data_storage[new_data.get_name()].popleft()
            self._time_storage[new_data.get_name()].popleft()
    
    def get_data_by_name(self, data_name: str)-> deque:
        return self._data_storage[data_name]
    
    def get_time_by_name(self, data_name: str)-> deque:
        return self._time_storage[data_name]
    
    def get_warmup_length(self)-> int:
        return self._warmup_length
    
    def get_signal_name(self)-> str:
        return self._signal_name
    
    @abstractmethod
    def calculate_signal(self)-> float:
        """
        Virtual method to be implemented by the derived class.
        """
        pass
    
class MASignal(SignalBase):
    """
    Moving average signal.
    """
    def __init__(self,
                input_data_array: List[Event],
                warmup_length: int,
                signal_name: str)-> None:
        super().__init__(input_data_array, warmup_length, signal_name)
    def calculate_signal(self)-> float:
        """
        Compute the moving average.
        """
        prices = self.get_data_by_name(self._input_data_array[0])
        return sum(prices) / len(prices)
    
class WMomSignal(SignalBase):
    """
    Weighted momentum signal.
    """
    def __init__(self,
            input_data_array: List[Event],
            warmup_length: int,
            signal_name: str)-> None:
        super().__init__(input_data_array, warmup_length, signal_name)
        # Constants
        self._trading_days_in_month = 21
        self._normalization_factor = 4.0
        self._pivot_months = [1, 3, 6, 12]
    def calculate_signal(self)-> float:
        """
        Compute the weighted momentum.
        """
        prices = self.get_data_by_name(self._input_data_array[0])
        weighted_momentum = 0.0
        for _month in self._pivot_months:
            p0 = prices[-1]
            pt = prices[len(prices) - (_month * _trading_days_in_month) - 1]
            weighted_momentum += (12.0 / _month) * (p0 / pt - 1.0) / self._normalization_factor
        return weighted_momentum

In [20]:
from typing import Dict

class StrategyBase(ABC):
    """
    Base class for strategy.
    """
    def __init__(self,
                input_signal_array: List[str],
                trade_combo_array: List[str],
                warmup_length: int,
                initial_capital: float = 100.0)-> None:
        """
        Initialize strategy base.
        
        Parameters
        ----------
        input_signal_array: List[str]
            List of event subscription that is required for trade decision.
        trade_combo_array: List[str]
            List of event subscription that is required for ordering.   
        warmup_length: int
            Number of data points to 'burn'
        initial_capital: float
            Initial capital.
        """
        self._warmup_length = warmup_length
        self._mtm = initial_capital
        self._strategy_active = True # A strategy is active by default
        self._initialize_signal_time_storage(input_signal_array)
        self._initialize_combo_time_storage(trade_combo_array)
        self._input_signal_array = input_signal_array
        self._trade_combo_array = trade_combo_array
        
        position_temp = {sec_name: 0.0 for sec_name in trade_combo_array}
        # Remember to add cash account
        position_temp['cash_account'] = initial_capital
        self._combo_positions = position_temp
        
        self._combo_order = {sec_name: 0.0 for sec_name in trade_combo_array}
        self._average_entry_price = {sec_name: 0.0 for sec_name in trade_combo_array}
        self._mtm_price = {sec_name: 0.0 for sec_name in trade_combo_array}
    
    def _initialize_signal_time_storage(self, input_signal_array: List[str])-> None:
        """
        Initialize storage.
        
        Parameters
        ----------
        input_signal_array: List[str]
            List of signal subscription that is required for ordering.
        """
        signal_storage = {}
        signal_time_storage = {}
        
        for input_name_ in input_signal_array:
            signal_storage[input_name_] = deque([])
            signal_time_storage[input_name_] = deque([])
        self._signal_storage = signal_storage
        self._signal_time_storage = signal_time_storage
        
    def _initialize_combo_time_storage(self, trade_combo_array: List[str])-> None:
        """
        Initialize storage.
        
        Parameters
        ----------
        trade_combo_array: List[str]
            List of constituents in the combo.
        """
        combo_time_storage = {}
        
        for input_name_ in trade_combo_array:
            combo_time_storage[input_name_] = deque([])
        self._combo_time_storage = combo_time_storage
        
    def get_strategy_active(self)-> bool:
        return self._strategy_active
        
    def set_strategy_active(self, activity: bool)-> None:
        self._strategy_active = activity
    
    def check_all_signals_received(self, signal_name: str)-> bool:
        output = True
        num_signal = len(self._input_signal_array)
        if num_signal == 1:
            return output # If there is only one incoming signal stream, no need to check
        for i in range(num_signal):
            event_name_i = self._input_signal_array[i]
            if len(self.get_signal_time_by_name(event_name_i)) == 0:
                return False
            
            time_diff = self.get_signal_time_by_name(signal_name)[-1] - self.get_signal_time_by_name(event_name_i)[-1]
            if time_diff > timedelta(microseconds = 1):
                return False
        return output
    
    def get_signal_time_by_name(self, signal_name: str)-> deque:
        return self._signal_time_storage[signal_name]
    
    def check_all_combo_data_received(self, data_name: str)-> bool:
        output = True
        num_data = len(self._trade_combo_array)
        if num_data == 1:
            return output # If there is only one incoming data stream, no need to check
        for i in range(num_data):
            event_name_i = self._trade_combo_array[i]
            if len(self.get_combo_time_by_name(event_name_i)) == 0:
                return False
            
            time_diff = self.get_combo_time_by_name(data_name)[-1] - self.get_combo_time_by_name(event_name_i)[-1]
            if time_diff > timedelta(microseconds = 1):
                return False
        return output
    
    def get_combo_time_by_name(self, data_name: str)-> deque:
        return self._combo_time_storage[data_name]
    
    @abstractmethod
    def make_order_decision(self)-> Dict[str, float]:
        """
        Virtual method to be implemented by the derived class.
        """
        pass

In [11]:
class DataSlot:
    """
    Class for data slot.
    """
    def __init__(self,
                data_name: str,
                parent_signals: List[SignalBase])-> None:
        """
        Initialize data slot.

        Parameters
        ----------
        data_name: str
            Name of the input data.
        parent_signals: List[SignalBase]
            Signals that listens to this data.
        """
        self._data_name = data_name
        self._parent_signals = parent_signals

    def on_event(self, new_data: TimeDouble)-> None:
        """
        Perform the action upon getting an event.

        When there is an event (arrival of data) we want to
        - Update data storage
        - Calculate signal if warming up is complete
        
        Parameters
        ----------
        new_data: TimeDouble
            Incoming new data.
        """
        for _parent in self._parent_signals:
            # 1. Update data storage
            _parent.update_data(new_data)

            # 2. If warming up is complete and all data arrived, calculate and publish signal
            if len(_parent.get_data_by_name(new_data.get_name())) == _parent.get_warmup_length() and _parent.check_all_received(new_data.get_name()):
                signal = _parent.calculate_signal()
                ### DEBUG
                print(signal)
                latest_timestamp = _parent.get_time_by_name(new_data.get_name())[-1]
                signal_event = Event(name = _parent.get_signal_name())
                signal_event.emit(TimeDouble(_parent.get_signal_name(), latest_timestamp, signal))

In [8]:
class EventContext:
    """
    Class for event context.
    TBD: Make it a singleton
    """
    def __init__(self, broker_handle: ibi.IB, data_handle: ibi.IB)-> None:
        """
        Initialize event context.
        
        Parameters
        ----------
        broker_handle: ibi.IB
            Broker API handle.
        data_handle: ibi.IB
            Data source handle.
        """
        self._broker_handle = broker_handle
        self._data_handle = data_handle
    def get_broker(self)-> ibi.IB:
        return self._broker_handle
    def get_data(self)-> ibi.IB:
        return self._data_handle
    
class BrokerEventRelay:
    """
    Class for broker event relay.
    TBD: Allow for different brokers (create derived classes)
    TBD: Add different relay member functions (open, high, low, close, volume)
    """
    def __init__(self,
                 listener: DataSlot,
                 data_name: str,
                 field_name: str = 'close',
                 )-> None:
        """
        Initialize broker event relay.
        
        Parameters
        ----------
        listener: DataSlot
            Listener data slot.
        data_name: str
            Name of the input data.
        field_name: str
            Data field name (open, high, low, close, volume, etc.). 
        """
        self._relay_event = Event()
        self._field_name = field_name
        self._relay_event += listener.on_event
        self._data_name = data_name
        
    def ib_live_bar(self,
                    bars: ibi.RealTimeBarList,
                    has_new_bar: bool)-> None:
        """
        Translate IB real time bar event into price update.
        
        Parameters
        ----------
        bars: ibi.RealTimeBarList
            IB RealTimeBarList.
        has_new_bar: bool
            Whether there is new bar.
        """
        if has_new_bar:
            if self._field_name == 'close':
                field = bars[-1].close
            else:
                raise TypeError('BrokerEventRelay.ib_live_bar: Unsupported data field type.')
            relay_data = TimeDouble(self._data_name, bars[-1].time, field)
            self._relay_event.emit(relay_data)

In [9]:
ibi.util.startLoop()
                
# Connection
ib = ibi.IB()
ib.connect('127.0.0.1', 4002, clientId=13)

<IB connected to 127.0.0.1:4002 clientId=13>

In [12]:
my_ec = EventContext(ib, ib)

# Subscription
my_contract = ibi.Forex('EURUSD')   
live_bars = ib.reqRealTimeBars(my_contract, 5, 'MIDPOINT', False)

# Create signal
input_data_name = 'EURUSD_close'
my_ma_signal = MASignal([input_data_name], 3, "EURUSD_MA")
my_eurusd_data = DataSlot(input_data_name, [my_ma_signal])
my_ber = BrokerEventRelay(my_eurusd_data, input_data_name)
live_bars.updateEvent += my_ber.ib_live_bar

ib.sleep(40)
ib.cancelRealTimeBars(live_bars)

1.17176
1.171755
1.1717666666666666
1.17178
1.1717816666666667
1.1717783333333334


Peer closed connection.


In [ ]:
ib.disconnect()